In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/DataTesi')
df

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,iid,datadate,cusip,cheqvm,dvrate,cshtrm,prccm,prchm,prclm,trfm,trt1m,ccmbegdt
0,1000,P,1,LU,25881,23369,19701113,19780630,1,19701130,000032102,NaN,NaN,31800.0,9.0000,10.875,8.5000,1.0,-28.0000,19691201
1,1000,P,1,LU,25881,23369,19701113,19780630,1,19701231,000032102,NaN,NaN,64100.0,10.0000,10.500,7.5000,1.0,11.1111,19700101
2,1000,P,1,LU,25881,23369,19701113,19780630,1,19710131,000032102,NaN,NaN,63600.0,9.0000,10.125,8.6250,1.0,-10.0000,19700201
3,1000,P,1,LU,25881,23369,19701113,19780630,1,19710228,000032102,NaN,NaN,215500.0,11.1250,11.375,9.1250,1.0,23.6111,19700301
4,1000,P,1,LU,25881,23369,19701113,19780630,1,19710331,000032102,NaN,NaN,206200.0,9.7500,12.875,9.6250,1.0,-12.3596,19700401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852986,351038,P,90,LC,16161,55612,20211029,E,90,20220630,74907L201,NaN,NaN,57153170.0,0.4503,0.940,0.3234,1.0,-29.0866,20210701
3852987,351038,P,90,LC,16161,55612,20211029,E,90,20220731,74907L201,NaN,NaN,92595961.0,0.3567,0.890,0.3313,1.0,-20.7861,20210801
3852988,351038,P,90,LC,16161,55612,20211029,E,90,20220831,74907L201,NaN,NaN,102416363.0,3.2700,28.110,3.1200,1.0,-26.6611,20210901
3852989,351038,P,90,LC,16161,55612,20211029,E,90,20220930,74907L201,NaN,NaN,52187902.0,2.2300,5.200,2.2108,1.0,-31.8043,20211001


In [3]:
price_volume_df = df.groupby(['GVKEY', 'datadate'])[['prccm', 'cshtrm']].mean().reset_index()
price_volume_df

,GVKEY,datadate,prccm,cshtrm
0,1000,19701130,9.0000,31800.0
1,1000,19701231,10.0000,64100.0
2,1000,19710131,9.0000,63600.0
3,1000,19710228,11.1250,215500.0
4,1000,19710331,9.7500,206200.0
...,...,...,...,...
3820630,351038,20220630,0.4503,57153170.0
3820631,351038,20220731,0.3567,92595961.0
3820632,351038,20220831,3.2700,102416363.0
3820633,351038,20220930,2.2300,52187902.0


In [133]:
prices = price_volume_df.pivot_table(values='prccm', index='datadate', columns='GVKEY')
volumes = price_volume_df.pivot_table(values='cshtrm', index='datadate', columns='GVKEY')
prices = prices.loc[:, volumes.columns]

In [134]:
na_columns_filter = volumes.isna().mean(axis=0) < .75
na_columns_filter = (1 - volumes.isna()).sum(axis=0) > 120
na_filt_prices = prices.loc[:, na_columns_filter]
na_filt_volumes = volumes.loc[:, na_columns_filter]

In [135]:
price_columns_filter = (na_filt_prices.min() > 5) & (na_filt_prices.max() < 10**5)
price_filt_prices = na_filt_prices.loc[:, price_columns_filter]
price_filt_volumes = na_filt_volumes.loc[:, price_columns_filter]

In [136]:
log_volumes = np.log10(price_filt_volumes)
volume_columns_filter = (log_volumes - log_volumes.mean(axis=1).values.reshape((-1,1)) + 
                         log_volumes.iloc[-1].mean()).mean() > 6
volume_filt_prices  = price_filt_prices.loc[:, volume_columns_filter]
volume_filt_volumes = price_filt_volumes.loc[:, volume_columns_filter]

In [137]:
volume_filt_prices

GVKEY,1010,1034,1040,1054,1062,1075,1076,1078,1119,1128,...,264647,264686,264708,266214,268208,269005,277812,287462,287882,294524
datadate,,,,,,,,,,,,,,,,,,,,,
19620630,59.00,NaN,22.250,NaN,NaN,25.75,NaN,63.25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19620731,67.48,NaN,21.000,NaN,NaN,28.00,NaN,63.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19620831,67.12,NaN,21.750,NaN,NaN,28.75,NaN,58.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19620930,63.48,NaN,19.500,NaN,NaN,28.00,NaN,60.25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19621031,68.00,NaN,18.125,NaN,NaN,26.63,NaN,64.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220630,NaN,NaN,NaN,NaN,14.67,73.12,16.50,108.65,15.34,NaN,...,8.8000,25.31,NaN,63.59,15.12,NaN,11.88,7.28,10.86,87.46
20220731,NaN,NaN,NaN,NaN,15.13,73.47,18.42,108.84,16.64,NaN,...,9.3900,27.55,NaN,73.69,16.34,NaN,8.88,7.19,10.74,89.12
20220831,NaN,NaN,NaN,NaN,13.99,75.35,18.54,102.65,16.07,NaN,...,9.1300,26.65,NaN,71.38,16.45,NaN,7.77,7.29,10.31,83.00


In [138]:
volume_filt_prices.to_csv('data/prices_CRSP.csv')
volume_filt_volumes.to_csv('data/volumes_CRSP.csv')